In [1]:
%matplotlib inline

In [2]:
# Import sim functions
from neurodsp.sim import sim_powerlaw, sim_random_walk, sim_synaptic_current
from neurodsp.sim import sim_oscillation
from neurodsp.utils import set_random_seed

# Import function to compute power spectra
from neurodsp.spectral import compute_spectrum

# Import utilities for plotting data
from neurodsp.utils import create_times
from neurodsp.plts.spectral import plot_power_spectra
from neurodsp.plts.time_series import plot_time_series

import matplotlib.pyplot as plt
import ipywidgets
import numpy as np 

In [3]:
# Set the random seed, for consistency simulating data
set_random_seed(0)


# PSD of periodic Signal

In [4]:
def plot_inter(frequency): 

    # Set some general settings, to be used across all simulations
    fs = 100 * frequency
    n_seconds = 5
    # Create a times vector for the simulations
    times = create_times(n_seconds, fs)
    
    # Simulate a sinusoidal oscillation
    osc_sine = sim_oscillation(n_seconds, fs, frequency, cycle='sine')

    # Plot the simulated data, in the time domain
    plot_time_series(times, osc_sine)

    # compute PSD of combined signal
    freqs, psd = compute_spectrum(osc_sine, fs)

    # plot frequqncy PSD
    #plt.figure()
    #plt.plot(freqs, psd)
    #plt.plot(freqs, psd)
    #plt.yscale('log')
    #plt.xscale('log')
    #plt.xlim(0,100)

    # PLot logged PSD
    plot_power_spectra(freqs, psd, title='Power-spectral Density')
    

slider_fre = ipywidgets.FloatSlider(value=10,min=1,max=50,step=1)
ipywidgets.interact(plot_inter, frequency=slider_fre)

interactive(children=(FloatSlider(value=10.0, description='frequency', max=50.0, min=1.0, step=1.0), Output())…

<function __main__.plot_inter(frequency)>

# PSD of colored Noise

In [5]:
def plot_inter(exponent):

    # Set some general settings, to be used across all simulations
    fs = 1000
    n_seconds = 100
    # Create a times vector for the simulations
    times = create_times(n_seconds, fs)

    # generate some aperiodic signal 
    noise = sim_powerlaw(n_seconds, fs, exponent)
    plot_time_series(times, noise, title='Colored Noise')
    
    # compute PSD of combined signal
    freqs, psd = compute_spectrum(noise, fs)

    # plot frequqncy PSD
    #plt.figure()
    #plt.plot(freqs,psd)
    #plt.xlim(0,100)

    # PLot logged PSD
    plot_power_spectra(freqs, psd, title='Power-spectral Density')

slider_exp = ipywidgets.FloatSlider(value=0,min=-4,max=0,step=1)

ipywidgets.interact(plot_inter, exponent=slider_exp)




interactive(children=(FloatSlider(value=0.0, description='exponent', max=0.0, min=-4.0, step=1.0), Output()), …

<function __main__.plot_inter(exponent)>

# Simulating Aperiodic Signals

In this interactive figure, you can simulate neural data with 2 peak frequencies and some aperiodic component. 

In the PSD plot at the end you can see an 1/f-like activity, whereby power decreases linearly across
increasing frequencies, when plotted in log-log (depending on the choosen exponent).

Let's start with a power law signal, specifically a brown noise process, or a signal
for which the power spectrum is distributed as 1/f^2.

In [6]:
def plot_inter(exponent,freq1,freq2):
    
    # Set some general settings, to be used across all simulations
    fs = 10 * freq1 * freq2
    n_seconds = 10
    # Create a times vector for the simulations
    times = create_times(n_seconds, fs)

    # generate some aperiodic brown noise 
    # Simulate brown noise powerlaw activity
    noise = sim_powerlaw(n_seconds, fs, exponent)
    plot_time_series(times, noise, title='Colored Noise')
    
    
    # Simulate a sinusoidal oscillation
    osc_sine1 = sim_oscillation(n_seconds, fs, freq1, cycle='sine')
    osc_sine2 = sim_oscillation(n_seconds, fs, freq2, cycle='sine')
    osc_sign = osc_sine1 + osc_sine2
    plot_time_series(times, osc_sign, title='Periodic signal')
    
    # combine signal
    comb_sign = noise + osc_sign
    plot_time_series(times, comb_sign, title='Combined signal')
    
    # compute PSD of combined signal with noise
    freqs_comb, psd_comb = compute_spectrum(comb_sign, fs)
    
    # compute PSD of combined signal without noise
    freqs_osc, psd_osc = compute_spectrum(osc_sign, fs)
    
    plot_power_spectra(freqs_comb, psd_comb, title='Power-spectral Density of combined Signal')
    #plot_power_spectra([freqs_comb,freqs_osc], [psd_comb,psd_osc], title='Power-spectral Density')
    #plt.legend(["Combined", "Oscillator"])
    
    
slider_exp = ipywidgets.FloatSlider(value=0,min=-5,max=0,step=1)
slider_freq1 = ipywidgets.FloatSlider(value=10,min=1,max=50,step=1)
slider_freq2 = ipywidgets.FloatSlider(value=13,min=1,max=50,step=1)

ipywidgets.interact(plot_inter, exponent=slider_exp, freq1 = slider_freq1, freq2 = slider_freq2)

interactive(children=(FloatSlider(value=0.0, description='exponent', max=0.0, min=-5.0, step=1.0), FloatSlider…

<function __main__.plot_inter(exponent, freq1, freq2)>

# Filter 1/f Activity

In [10]:
def plot_inter(f_hipass, f_lowpass, exponent):

    # Set some general settings, to be used across all simulations
    fs = 1000
    n_seconds = 10
    # Create a times vector for the simulations
    times = create_times(n_seconds, fs)

    # generate some aperiodic signal 
    noise = sim_powerlaw(n_seconds, fs, exponent)
    noise_filt = sim_powerlaw(n_seconds, fs, exponent,f_range=(f_lowpass, f_hipass))
    #noise_filt = sim_powerlaw(n_seconds, fs, exponent,f_range=(5, 10))

    plot_time_series(times, noise, title='Colored Noise')
    plot_time_series(times, noise_filt, title='Filtered Colored Noise')

    # compute PSD of combined signal
    freqs, psd = compute_spectrum(noise, fs)
    freqs_filt, psd_filt = compute_spectrum(noise_filt, fs)

    # plot frequqncy PSD
    plt.figure()
    #plt.plot(freqs,psd)
    plt.plot(freqs_filt,psd_filt)
    plt.xlim(1,50)
    #plt.ylim(0,0.05)
    plt.ylabel("Power")
    plt.xlabel("Frequency")
    
slider_exp = ipywidgets.FloatSlider(value=0,min=-10,max=0,step=1)
slider_low = ipywidgets.FloatSlider(value=5,min=1,max=9,step=1)
slider_high = ipywidgets.FloatSlider(value=10,min=10,max=20,step=1)

ipywidgets.interact(plot_inter, f_lowpass = slider_low, f_hipass = slider_high, exponent=slider_exp)




interactive(children=(FloatSlider(value=10.0, description='f_hipass', max=20.0, min=10.0, step=1.0), FloatSlid…

<function __main__.plot_inter(f_hipass, f_lowpass, exponent)>